##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Detecção de objetos com o TensorFlow Lite Model Maker

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/lite/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

Neste notebook do Colab, você verá como usar o [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) (criador de modelos do TF Lite) para treinar um modelo personalizado de detecção de objetos para detectar saladas em imagens em um dispositivo móvel.

A biblioteca Model Maker usa *aprendizado por transferência* para simplificar o processo de treinar um modelo do TensorFlow Lite usando um dataset personalizado. Retreinar um modelo do TensorFlow Lite com seu próprio dataset personalizado reduz a quantidade necessária de dados de treinamento e diminui o tempo de treinamento.

Você usará o dataset *Salads*, disponível publicamente, que foi criado usando o [Dataset Open Images V4](https://storage.googleapis.com/openimages/web/index.html).

Cada imagem do dataset contém objetos rotulados como uma das seguintes classes:

- Baked Good (Assado)
- Cheese (Queijo)
- Salad (Salada)
- Seafood (Fruto do mar)
- Tomato (Tomate)

O dataset contém os retângulos limítrofes que especificam onde cada objeto está localizado, juntamente com o rótulo do objeto.

Veja abaixo uma imagem de exemplo do dataset:

<br>


<img src="https://cloud.google.com/vision/automl/object-detection/docs/images/quickstart-preparing_a_dataset.png" width="400" hspace="0"> 


## Pré-requisitos


### Instale os pacotes obrigatórios

Comece instalando os pacotes obrigatórios, incluindo o pacote do Model Maker disponível no [repositório do GitHub](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) e a biblioteca pycocotools, que será usada para avaliação.

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools
!pip install -q opencv-python-headless==4.1.2.30
!pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0

Importe os pacotes necessários.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### Prepare o dataset

Você usará o mesmo dataset que o usado no [Guia de início rápido](https://cloud.google.com/vision/automl/object-detection/docs/edge-quickstart#preparing_a_dataset) do AutoML.

O dataset *Salads* está disponível em: `gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv`.

Ele contém 175 imagens para treinamento, 25 para validação e 25 para teste. O dataset tem cinco classes: `Salad` (Salada), `Seafood` (Fruto do mar), `Tomato` (Tomate), `Baked goods` (Assado), `Cheese` (Queijo).

<br>

O dataset é fornecido no formato CSV:

```
TRAINING,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Salad,0.0,0.0954,,,0.977,0.957,,
VALIDATION,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Seafood,0.0154,0.1538,,,1.0,0.802,,
TEST,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Tomato,0.0,0.655,,,0.231,0.839,,
```

- Cada linha corresponde a um objeto localizado dentro de uma imagem maior, em que cada objeto é designado especificamente como dados de teste, treinamento ou validação. Você entenderá melhor o que isso significa posteriormente neste notebook.
- As três linhas incluídas aqui indicam **três objetos distintos localizados dentro da mesma imagem** disponível em `gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg`.
- Cada linha tem um rótulo diferente: `Salad`, `Seafood`, `Tomato`, etc.
- Os retângulos limítrofes são especificados para cada imagem usando os vértices superior esquerdo e inferior direito.

Veja abaixo essas três linhas:

<br>


<img src="https://cloud.google.com/vision/automl/object-detection/docs/images/quickstart-preparing_a_dataset.png" width="400" hspace="100">

Se você quiser saber mais sobre como preparar seu próprio arquivo CSV e quais são os requisitos mínimos para criar um dataset válido, confira mais detalhes no guia [Como preparar seus dados de treinamento](https://cloud.google.com/vision/automl/object-detection/docs/prepare).

Se você for iniciante no Google Cloud, talvez se pergunte o que a URL `gs://` significa. É uma URL de arquivos armazenados no [Google Cloud Storage](https://cloud.google.com/storage) (GCS). Se você tornar públicos seus arquivos do GCS ou se [autenticar seu cliente](https://cloud.google.com/storage/docs/authentication#libauth), o Model Maker poderá ler esses arquivos de maneira similar a arquivos locais.

Mas você não precisa manter as imagens no Google Cloud para usar o Model Maker. É possível usar um caminho local no arquivo CSV, e o Model Maker funcionará normalmente.

## Início rápido

Há seis etapas para treinar um modelo de detecção de objetos:

**Etapa 1 – Escolha uma arquitetura de modelo de detecção de objetos**

O modelo usado neste tutorial é EfficientDet-Lite0. EfficientDet-Lite[0-4] é uma família de modelos de detecção de objetos para dispositivos móveis/otimizados para IoT, derivados da arquitetura [EfficientDet](https://arxiv.org/abs/1911.09070).

Veja abaixo a comparação do desempenho entre os modelos de EfficientDet-Lite.

Arquitetura do modelo | Tamanho (em MB)* | Latência (em ms)** | Precisão média***
--- | --- | --- | ---
EfficientDet-Lite0 | 4,4 | 37 | 25,69%
EfficientDet-Lite1 | 5,8 | 49 | 30,55%
EfficientDet-Lite2 | 7,2 | 69 | 33,97%
EfficientDet-Lite3 | 11,4 | 116 | 37,70%
EfficientDet-Lite4 | 19,9 | 260 | 41,96%

<i> *Tamanho dos modelos quantizados em inteiros. <br> **Latência medida no Pixel 4 usando 4 threads da CPU. <br> ***A precisão média é a mAP no dataset de validação COCO 2017.</i>


In [ ]:
spec = model_spec.get('efficientdet_lite0')

**Etapa 2 – Carregue o dataset**

O Model Maker recebe os dados de entrada no formato CSV. Use o método `object_detector.DataLoader.from_csv` para carregar o dataset e dividi-lo em imagens de treinamento, validação e teste.

- Imagens de treinamento: são usadas para treinar o modelo de detecção de objetos para reconhecer ingredientes de saladas.
- Imagens de validação: são as imagens que o modelo não viu durante o processo de treinamento. Elas serão usadas para decidir quando interromper o treinamento para evitar o [overfitting](https://en.wikipedia.org/wiki/Overfitting).
- Imagens de teste: são usadas para avaliar o desempenho do modelo final.

Você pode carregar o arquivo CSV diretamente do Google Cloud Storage, mas não precisa manter as imagens no Google Cloud para usar o Model Maker. É possível especificar um arquivo CSV local em seu computador, e o Model Maker funcionará normalmente.

In [ ]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv')

**Etapa 3 – Treine o modelo do TensorFlow com os dados de treinamento**

- O modelo EfficientDet-Lite0 usa `epochs = 50` por padrão, ou seja, ele percorre o dataset de treinamento 50 vezes. Você pode avaliar a exatidão de validação durante o treinamento e interromper antecipadamente para evitar o overfitting.
- Defina `batch_size = 8` aqui para ver que demora 21 passos para percorrer as 175 imagens no dataset de treinamento.
- Defina `train_whole_model=True` para fazer os ajustes finos do modelo inteiro em vez de apenas treinar a camada head para aumentar a exatidão. A contrapartida é que pode demorar mais tempo para treinar o modelo.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data)

**Etapa 4 – Avalie o modelo com os dados de teste**

Após treinar o modelo de detecção de objetos usando as imagens do dataset de treinamento, use as 25 imagens restantes no dataset de teste para avaliar o desempenho do modelo com os novos dados que ele nunca viu.

Como o tamanho padrão do lote é 64, vai demorar 1 passo para percorrer as 25 imagens do dataset de teste.

As métricas de avaliação são as mesmas que as de [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
model.evaluate(test_data)

**Etapa 5 – Exporte para um modelo do TensorFlow Lite**

Para exportar o modelo treinado de detecção de objetos para o formato do TensorFlow Lite, basta especificar para qual pasta você deseja exportar o modelo quantizado. A técnica padrão de quantização pós-treinamento é a quantização completa em inteiros.

In [ ]:
model.export(export_dir='.')

**Etapa 6 – Avalie o modelo do TensorFlow Lite**

Diversos fatores podem afetar a exatidão do modelo ao exportar para o TF Lite:

- A [quantização](https://www.tensorflow.org/lite/performance/model_optimization) ajuda a reduzir o tamanho do modelo em até quatro vezes, e o custo é uma pequena redução da exatidão.
- O modelo original do TensorFlow usa [supressão não máxima (NMS, na sigla em inglês)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) por classe para pós-processamento, enquanto o modelo do TF Lite usa NMS global, que é muito mais rápido, mas menos exato. O Keras gera no máximo 100 detecções, enquanto o tflite gera no máximo 25 detecções.

Portanto, você precisará avaliar o modelo do TF Lite exportado e comparar a exatidão com o modelo original do TensorFlow.

In [ ]:
model.evaluate_tflite('model.tflite', test_data)

Você pode baixar o arquivo do modelo do TensorFlow Lite pela barra lateral esquerda do Colab. Clique com o botão direito no arquivo  `model.tflite` e selecione `Download` (Baixar) para baixá-lo para seu computador local.

Este modelo pode ser integrado a um aplicativo para Android ou iOS usando a [API ObjectDetector](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) da [biblioteca Task TensorFlow Lite](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview).

Confira mais detalhes de como o modelo é usado em um aplicativo funcional no [exemplo de aplicativo para detecção de objetos do TF Lite](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/android).

*Observação: o Android Studio Model Binding ainda não oferece suporte à detecção de objetos, então use a TensorFlow Lite Task Library.*

## Teste o modelo do TF Lite com suas imagens (opcional)

Você pode testar o modelo do TF Lite treinado usando imagens da Internet.

- Substitua `INPUT_IMAGE_URL` abaixo pela imagem de entrada desejada.
- Ajuste `DETECTION_THRESHOLD` para alterar a sensibilidade do modelo. Com um limiar mais baixo, o modelo identificará mais objetos, mas também haverá mais falsos positivos. Já com um limiar mais alto, o modelo só identificará objetos que detectou com alta confiança.

Embora seja necessário código boilerplate para executar o modelo no Python atualmente, só são necessárias algumas linhas de código para integrar o modelo a um aplicativo para dispositivos móveis.

In [ ]:
#@title Load the trained TFLite model and define some visualization functions

import cv2

from PIL import Image

model_path = 'model.tflite'

# Load the labels into a list
classes = ['???'] * model.model_spec.config.num_classes
label_map = model.model_spec.config.label_map
for label_id, label_name in label_map.as_dict().items():
  classes[label_id-1] = label_name

# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""

  signature_fn = interpreter.get_signature_runner()

  # Feed the input image to the model
  output = signature_fn(images=image)

  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
#@title Run object detection and show the detection results

INPUT_IMAGE_URL = "https://storage.googleapis.com/cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg" #@param {type:"string"}
DETECTION_THRESHOLD = 0.3 #@param {type:"number"}

TEMP_FILE = '/tmp/image.png'

!wget -q -O $TEMP_FILE $INPUT_IMAGE_URL
im = Image.open(TEMP_FILE)
im.thumbnail((512, 512), Image.ANTIALIAS)
im.save(TEMP_FILE, 'PNG')

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    TEMP_FILE,
    interpreter,
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
Image.fromarray(detection_result_image)

## Compile para o Edge TPU (opcional)

Agora que você tem um modelo de EfficientDet-Lite quantizado, é possível compilar e implantar em um [Coral EdgeTPU](https://coral.ai/).

**Etapa 1 – Instale o EdgeTPU Compiler**

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler

**Etapa 2 – Selecione o número de Edge TPUs e compile**

O EdgeTPU tem 8 MB de SRAM para fazer cache dos parâmetros do modelo ([confira mais informações](https://coral.ai/docs/edgetpu/compiler/#parameter-data-caching)). Portanto, para modelos maiores do que 8 MB, o tempo de inferência aumentará para que os parâmetros do modelo sejam transferidos. Uma forma de evitar isso é criar [pipelines do modelo](https://coral.ai/docs/edgetpu/pipeline/), ou seja, dividir o modelo em segmentos que podem ter um Edge TPU dedicado, o que pode melhorar a latência consideravelmente.

A tabela abaixo pode ser usada como referência para o número de Edge TPUs a serem usadas. Modelos maiores não serão compilados para uma única TPU, pois os tensores intermediários não cabem na memória do chip.

Arquitetura do modelo | Nº mínimo de TPUs | Nº recomendado de TPUs
--- | --- | ---
EfficientDet-Lite0 | 1 | 1
EfficientDet-Lite1 | 1 | 1
EfficientDet-Lite2 | 1 | 2
EfficientDet-Lite3 | 2 | 2
EfficientDet-Lite4 | 2 | 3

In [ ]:
NUMBER_OF_TPUS =  1#@param {type:"number"}

!edgetpu_compiler model.tflite --num_segments=$NUMBER_OF_TPUS

**Etapa 3 – Baixe e execute o modelo**

Com os modelos compilados, agora eles podem ser executados em Edge TPUs para detecção de objetos. Primeiro, baixe o arquivo do modelo do TensorFlow Lite compilado pela barra lateral esquerda do Colab. Clique com o botão direito no arquivo `model_edgetpu.tflite` e selecione `Download` (Baixar) para baixá-lo para seu computador local.

Agora você pode executar o modelo da maneira que desejar. Veja alguns exemplos de detecção:

- [Detecção com Pycoral](https://github.com/google-coral/pycoral/blob/master/examples/detect_image.py)
- [Detecção básica com o TF Lite](https://github.com/google-coral/tflite/tree/master/python/examples/detection)
- [Exemplo de detecção de vídeo](https://github.com/google-coral/examples-camera)
- [API libcoral do C++](https://github.com/google-coral/libcoral)


## Uso avançado

Esta seção abrange tópicos de uso avançado, como ajustar o modelo e os hiperparâmetros de treinamento.

### Carregue o dataset

#### Carregue seus próprios dados

Você pode carregar seu próprio dataset neste tutorial. Para carregá-lo, use a barra lateral esquerda no Colab.


<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_object_detection.png" width="1000" hspace="0" alt="Upload File">

Se você preferir não carregar o dataset na nuvem, pode executar a biblioteca localmente de acordo com este [guia](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

#### Carregue seus dados com um formato de dados diferente

A biblioteca do Model Maker também oferece suporte ao método `object_detector.DataLoader.from_pascal_voc` para carregar dados com o formato [PASCAL VOC](https://towardsdatascience.com/coco-data-format-for-object-detection-a4c5eaf518c5#:~:text=Pascal%20VOC%20is%20an%20XML,for%20training%2C%20testing%20and%20validation). [makesense.ai](https://www.makesense.ai/) e [LabelImg](https://github.com/tzutalin/labelImg) são as ferramentas que podem fazer anotações na imagem e salvá-las como arquivos XML no formato de dados PASCAL VOC:

```python
object_detector.DataLoader.from_pascal_voc(image_dir, annotations_dir, label_map={1: "person", 2: "notperson"})
```


### Personalize os hiperparâmetros do modelo EfficientDet

Os parâmetros do modelo e do pipeline de treinamento que podem ser ajustados são:

- `model_dir`: local onde salvar os arquivos de checkpoint do modelo. Caso não seja definido, será usado um diretório temporário.
- `steps_per_execution`: número de passos por execução de treinamento.
- `moving_average_decay`: Float. Decaimento a ser usado para manter as médias móveis dos parâmetros treinados.
- `var_freeze_expr`: expressão regular para mapear o nome prefixo das variáveis a serem congeladas, ou seja, que não mudam durante o treinamento. Mais especificamente, use `re.match(var_freeze_expr, variable_name)` no codebase para mapear as variáveis a serem congeladas.
- `tflite_max_detections`: inteiro, 25 por padrão. Número máximo de detecções de saída no modelo do TF Lite.
- `strategy`: string que especifica qual estratégia de distribuição será usada. Aceita valores como 'tpu', 'gpus', None (nenhuma). tpu' significa que será usada TPUStrategy. 'gpus' significa que será usada MirroredStrategy para várias GPUs. Em caso de None, será usado o padrão do TF com OneDeviceStrategy.
- `tpu`: Cloud TPU a ser usada para treinamento. Deve ser o nome usado ao criar a Cloud TPU ou uma URL grpc://ip.address.of.tpu:8470.
- `use_xla`: usa XLA mesmo se a estratégia não for TPU. Se a estratégia for TPU, sempre usa XLA, e este sinalizador não tem efeito nenhum.
- `profile`: ativa o modo de profiling.
- `debug`: ativa o modo de depuração.

Outros parâmetros que podem ser ajustados são exibidos em [hparams_config.py](https://github.com/google/automl/blob/df451765d467c5ed78bbdfd632810bc1014b123e/efficientdet/hparams_config.py#L170).

Por exemplo: você pode definir `var_freeze_expr='efficientnet'`, que congela as variáveis com o prefixo de nome `efficientnet` (o padrão é `'(efficientnet|fpn_cells|resample_p6)')`. Isso permite que o modelo congele variáveis não treináveis e mantenha seus valores inalterados durante o treinamento.

```python
spec = model_spec.get('efficientdet_lite0')
spec.config.var_freeze_expr = 'efficientnet'
```

### Altere a arquitetura do modelo

Para alterar a arquitetura do modelo, basta mudar `model_spec`. Por exemplo: altere `model_spec` para o modelo EfficientDet-Lite4.

```python
spec = model_spec.get('efficientdet_lite4')
```

### Ajuste os hiperparâmetros de treinamento

A função `create` é a função que a biblioteca Model Maker usa para criar modelos. O parâmetro `model_spec` define a especificação do modelo. No momento, há suporte à classe `object_detector.EfficientDetSpec`. A função `create` consiste nas seguintes etapas:

1. Cria o modelo para detecção de objetos de acordo com `model_spec`.
2. Treina o modelo. O número padrão de épocas e do tamanho do lote é definido pelas variáveis `epochs` e `batch_size` no objeto `model_spec`. Além disso, você pode ajustar os hiperparâmetros de treinamento, como `epochs` e `batch_size`, que afetam a exatidão do modelo. Por exemplo:

- `epochs`: inteiro, 50 por padrão. Mais épocas podem levar a uma exatidão melhor, mas podem causar overfitting.
- `batch_size`: inteiro, 64 por padrão. Número de amostras a serem usadas em um passo de treinamento.
- `train_whole_model`: booleano, false (falso) por padrão. Se for igual a true (verdadeiro), treina o modelo inteiro. Caso contrário, treina somente as camadas que não coincidem com `var_freeze_expr`.

Por exemplo: você pode treinar com menos épocas e apenas a camada head. É possível aumentar o número de épocas para obter resultados melhores.

```python
model = object_detector.create(train_data, model_spec=spec, epochs=10, validation_data=validation_data)
```

### Exporte para diferentes formatos

Confira abaixo os formatos de exportação permitidos:

- `ExportFormat.TFLITE`
- `ExportFormat.LABEL`
- `ExportFormat.SAVED_MODEL`


Por padrão, é exportado somente o arquivo de modelo do  TensorFlow Lite que contém [metadados](https://www.tensorflow.org/lite/models/convert/metadata) para que você possa usá-lo posteriormente em um aplicativo de aprendizado de máquina em dispositivos. O arquivo de rótulos é incorporado aos metadados.

Em diversos aplicativos de aprendizado de máquina em dispositivos, o tamanho do modelo é um fator importante. Portanto, recomendamos aplicar quantização no modelo para deixá-lo menor e possivelmente mais rápido. Quanto aos modelos EfficientDet-Lite, a quantização completa em inteiros é usada para quantizar o modelo por padrão. Confira mais detalhes em [Quantização pós-treinamento](https://www.tensorflow.org/lite/performance/post_training_quantization).

```python
model.export(export_dir='.')
```

Você também pode optar por exportar outros arquivos relacionados ao modelo para avaliá-los melhor. Por exemplo: você pode exportar tanto o SavedModel quanto o arquivo de rótulos da seguinte forma:

```python
model.export(export_dir='.', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])
```

### Personalize a quantização pós-treinamento em um modelo do TensorFlow Lite

A [quantização pós-treinamento](https://www.tensorflow.org/lite/performance/post_training_quantization) é uma técnica de conversão que pode reduzir o tamanho do modelo e a latência de inferência, além de aumentar a velocidade de inferência da CPU e do acelerador de hardware com uma pequena redução da exatidão do modelo. A quantização é amplamente utilizada para otimizar o modelo.

A biblioteca Model Maker aplica uma técnica padrão de quantização pós-treinamento ao exportar o modelo. Se você quiser personalizar a quantização pós-treinamento, o Model Maker oferece suporte a diversas opções usando [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig). Vejamos a quantização de float 16 como exemplo. Primeiro, definimos a configuração de quantização.

```python
config = QuantizationConfig.for_float16()
```

Em seguida, exportamos o modelo do TensorFlow Lite com essa configuração.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# Saiba mais

Leia o exemplo de [detecção de objetos](https://www.tensorflow.org/lite/examples/object_detection/overview) para aprender os detalhes técnicos. Confira mais informações em:

- [Guia](https://www.tensorflow.org/lite/models/modify/model_maker) e [referência da API](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker) do TensorFlow Lite Model Maker.
- Task Library: [ObjectDetector](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) para implantação.
- Aplicativos de referência completos para [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/android), [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/ios) e [Raspberry PI](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/raspberry_pi).
